# <font color='blue'>**TP TEST DE CONNAISSANCES**</font>

# Importation des packages

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1) Instanciation de SparkSession

In [57]:
Spark = SparkSession.builder.master("local").appName("TestConnaissances").getOrCreate()

# Question 0 : Créations des dataframes 

In [56]:
l1 = [("0001","L'existentialisme est un humanisme","Philosophie"),("0002","Huis clos. Suivi de Les Mouches","Philosophie"),("0003","Mignonne allons voir si la rose","Poeme"), ("0004","Les Amours","Poeme")]
RDD1 = Spark.sparkContext.parallelize(l1) #Créé un RDD

book = RDD1.toDF(["bid","title","category"])
book.createOrReplaceTempView("bookSQL")
book.show()

+----+--------------------+-----------+
| bid|               title|   category|
+----+--------------------+-----------+
|0001|L'existentialisme...|Philosophie|
|0002|Huis clos. Suivi ...|Philosophie|
|0003|Mignonne allons v...|      Poeme|
|0004|          Les Amours|      Poeme|
+----+--------------------+-----------+



In [4]:
l2 = [("S15","toto","Math"),("S16","popo","Eco"),("S17","fofo","Mecanique")]
RDD2 = Spark.sparkContext.parallelize(l2) #Créé un RDD

Student = RDD2.toDF(["sid","sname","dept"])
Student.createOrReplaceTempView("StudentSQL")
Student.show()

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|     Math|
|S16| popo|      Eco|
|S17| fofo|Mecanique|
+---+-----+---------+



In [5]:
l3 = [("07890","0001"),("07890","0002"),("05678","0003"), ("05678","0003")]
RDD3 = Spark.sparkContext.parallelize(l3) #Créé un RDD

write = RDD3.toDF(["aid","bid"])
write.createOrReplaceTempView("writeSQL")
write.show()

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0003|
+-----+----+



In [6]:
l4 = [("07890","Jean Paul Sartre"), ("05678","Pierre de Ronsard")]
RDD4 = Spark.sparkContext.parallelize(l4) #Créé un RDD

Author = RDD4.toDF(["aid","name"])
Author.createOrReplaceTempView("AuthorSQL")
Author.show()

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



In [37]:
l5 = [("S15","0003","2020-01-02","2020-02-01"), ("S15","0002","2020-06-13","null"), ("S15","0001","2020-06-13","2020-10-13"), ("S16","0002","2020-01-24","2020-01-24"), ("S17","0001","2020-04-24","2020-07-24")]
RDD5 = Spark.sparkContext.parallelize(l5) #Créé un RDD

borrow = RDD5.toDF(["sid","bid","checkout_time","return_time"])
borrow.createOrReplaceTempView("borrowSQL")
borrow.show()

+---+----+-------------+-----------+
|sid| bid|checkout_time|return_time|
+---+----+-------------+-----------+
|S15|0003|   2020-01-02| 2020-02-01|
|S15|0002|   2020-06-13|       null|
|S15|0001|   2020-06-13| 2020-10-13|
|S16|0002|   2020-01-24| 2020-01-24|
|S17|0001|   2020-04-24| 2020-07-24|
+---+----+-------------+-----------+



# Question 1 : Trouvons les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [8]:
#DSL
borrow.join(Student,['sid'])\
    .join(book,['bid'])\
    .filter((F.col("sid") == "S15"))\
    .select("sname","sid","title")\
    .show()

+-----+---+--------------------+
|sname|sid|               title|
+-----+---+--------------------+
| toto|S15|Huis clos. Suivi ...|
| toto|S15|Mignonne allons v...|
| toto|S15|L'existentialisme...|
+-----+---+--------------------+



In [9]:
#SQL
Spark.sql(''' select S.sname, S.sid, b.title
              from borrowSQL as br join StudentSQL as S on br.sid=S.sid 
              join bookSQL as b on b.bid = br.bid
              where S.sid = "S15" 
              group by S.sname, S.sid, b.title          
              ''').show()

+-----+---+--------------------+
|sname|sid|               title|
+-----+---+--------------------+
| toto|S15|L'existentialisme...|
| toto|S15|Mignonne allons v...|
| toto|S15|Huis clos. Suivi ...|
+-----+---+--------------------+



# Question 2 : Trouvons les titres de tous les livres qui n'ont jamais été empruntés par un étudiant

In [10]:
#SQL
Spark.sql(''' select b.bid,br.bid,b.title
              from bookSQL as b Full Join borrowSQL as br on b.bid = br.bid 
              where br.bid is null
              ''').show()

+----+----+----------+
| bid| bid|     title|
+----+----+----------+
|0004|null|Les Amours|
+----+----+----------+



In [11]:
#DSL
book.join(borrow,['bid'],"left_anti")\
    .select("bid","title")\
    .show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



# Question 3 : Trouvons tous les étudiants qui ont emprunté le livre bid=’0002’

In [12]:
#DSL
borrow.join(Student,['sid'])\
    .filter((F.col("bid") == "0002"))\
    .select("sname","bid")\
    .show()

+-----+----+
|sname| bid|
+-----+----+
| popo|0002|
| toto|0002|
+-----+----+



In [13]:
#SQL
Spark.sql(''' select S.sname, br.bid
              from borrowSQL as br join StudentSQL as S on br.sid=S.sid 
              join bookSQL as b on b.bid = br.bid
              where br.bid = "0002"        
              ''').show()

+-----+----+
|sname| bid|
+-----+----+
| popo|0002|
| toto|0002|
+-----+----+



# Question 4 : Trouvons les titres de tous les livres empruntés par des étudiants en Mécanique 

In [14]:
#DSL
#On laisse dept dans l'affichage comme preuve
borrow.join(Student,['sid'])\
    .join(book,['bid'])\
    .filter((F.col("dept") == "Mecanique"))\
    .select("title", "dept")\
    .distinct()\
    .show()

+--------------------+---------+
|               title|     dept|
+--------------------+---------+
|L'existentialisme...|Mecanique|
+--------------------+---------+



In [15]:
#SQL
#On laisse dept dans l'affichage comme preuve
Spark.sql(''' select distinct b.title, S.dept
              from borrowSQL as br join StudentSQL as S on br.sid=S.sid 
              join bookSQL as b on b.bid = br.bid
              where S.dept ='Mecanique'        
              ''').show()

+--------------------+---------+
|               title|     dept|
+--------------------+---------+
|L'existentialisme...|Mecanique|
+--------------------+---------+



# Question 5 : Trouvons les étudiants qui n’ont jamais emprunté de livre.

In [16]:
#DSL
Student.join(borrow,['sid'],"left_anti")\
    .select("sname")\
    .show()

+-----+
|sname|
+-----+
+-----+



In [17]:
#SQL
Spark.sql(''' select S.sname
              from borrowSQL as br Full Join StudentSQL as S on br.sid = S.sid 
              where S.sname is null
              ''').show()

+-----+
|sname|
+-----+
+-----+



# Question 6 : Déterminer l’auteur qui a écrit le plus de livres

In [18]:
#SQL
#Récuperer les noms puis compter
Spark.sql(''' select A.name, count(*) as nb_vente
              from writeSQL as W Inner Join borrowSQL as br on W.bid = br.bid
              Inner Join AuthorSQL as A on W.aid = A.aid
              group by A.name
              order by nb_vente desc 
              ''').show(1)

+----------------+--------+
|            name|nb_vente|
+----------------+--------+
|Jean Paul Sartre|       4|
+----------------+--------+
only showing top 1 row



In [19]:
### DSL
write.join(borrow,['bid'],"inner")\
    .join(Author,['aid'],"inner")\
    .groupBy(F.col("name")) \
    .agg(F.count("*").alias("nb_vente")) \
    .orderBy(F.col("nb_vente"), ascending = False)\
    .show(1)


+----------------+--------+
|            name|nb_vente|
+----------------+--------+
|Jean Paul Sartre|       4|
+----------------+--------+
only showing top 1 row



# Question 7 : Déterminons les personnes qui n’ont pas encore rendu les livres.

In [20]:
#SQL
Spark.sql(''' select S.sname
              from borrowSQL as br join StudentSQL as S on br.sid=S.sid 
              where br.return_time = "null"        
              ''').show()

+-----+
|sname|
+-----+
| toto|
+-----+



In [21]:
#DSL
Student.join(borrow,['sid'])\
    .filter(F.col("return_time") == "null")\
    .select("sname")\
    .show()

+-----+
|sname|
+-----+
| toto|
+-----+



# Question 8 : Créons une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois, sinon 0. 

In [39]:
#SQL
#On compare par rapport au jour actuel pour les livres non rendus
Spark.sql(''' select sid, bid, checkout_time, return_time,
            case  
                  when datediff(return_time, checkout_time) < 90 then 0 
                  when datediff(return_time, checkout_time) >= 90 then 1 
                  when datediff(return_time, checkout_time) is null then
                      case 
                          when datediff(now(), checkout_time) < 90 then 0 else 1 
                      end 
            end as morethan3month
                     
                 
              from borrowSQL 
              ''').show()

+---+----+-------------+-----------+--------------+
|sid| bid|checkout_time|return_time|morethan3month|
+---+----+-------------+-----------+--------------+
|S15|0003|   2020-01-02| 2020-02-01|             0|
|S15|0002|   2020-06-13|       null|             1|
|S15|0001|   2020-06-13| 2020-10-13|             1|
|S16|0002|   2020-01-24| 2020-01-24|             0|
|S17|0001|   2020-04-24| 2020-07-24|             1|
+---+----+-------------+-----------+--------------+



In [78]:
Export = borrow.withColumn("morethan3month", F.when(F.datediff(F.col("return_time"), F.col("checkout_time")) < 90,0) 
                  .when(F.datediff(F.col("return_time"), F.col("checkout_time")) > 90,1)
                  .when(F.col("return_time") == "null" ,
                        F.when(F.datediff(F.current_date(), F.col("checkout_time")) < 90,0)
                        .otherwise(1)))\
      .select("sid","bid","checkout_time","return_time","morethan3month")
Export.show()

+---+----+-------------+-----------+--------------+
|sid| bid|checkout_time|return_time|morethan3month|
+---+----+-------------+-----------+--------------+
|S15|0003|   2020-01-02| 2020-02-01|             0|
|S15|0002|   2020-06-13|       null|             1|
|S15|0001|   2020-06-13| 2020-10-13|             1|
|S16|0002|   2020-01-24| 2020-01-24|             0|
|S17|0001|   2020-04-24| 2020-07-24|             1|
+---+----+-------------+-----------+--------------+



## Exportation

In [86]:
Export.write.format("csv").save("contention") # Exportation à ne lancer qu'une fois
# A chaque execution de cette ligne, le dossier nommé exported doit être supprimé.

<font color='green'>***Merci et bonne continuation***</font>